# Import needed dependencies

In [1]:
import urllib.request
import json
import re

import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib widget

import threading
import time

# Setup Controlling Variables

Start JuMonC and copy a scope token of at least `compute_data` scope. Fill i nthis token and adjust the baseurl if you are not running on the same port or not on localhost and fill in a correct token.

In [2]:
scopeToken = "dd48f3a0da63ffed740157e2f14a5573877bdff875027da8"
baseurl = "http://localhost:12121/"
networkPath = "v1/network/status/"

max_members = 31
timeStep = 2.0

data = None
lastData = None
startTime = None
dataLock = threading.Lock()

output = widgets.Output()
paused = False
linePlot = True
fig = None

# Query JuMonC network capabilities

In [3]:
with urllib.request.urlopen(baseurl + networkPath + "?token=" + scopeToken) as response:
    links = []
    text = response.read()
    text = text.decode("utf8", 'ignore')
    textDic = json.loads(text)
        
    for element in textDic:
        try:
            links.append(element["link"].split("/")[-1])
        except:
            continue
          
    del links[0]
    dataTypes = links
    
    interfaceDes = textDic[0]["parameters"][1]["description"]
    interfacesText = re.search('\[(.*)\]', interfaceDes).group(1)
    interfaces = interfacesText.split(", ")
    
    for i in range(len(interfaces)):
        interfaces[i] = interfaces[i][1:-1]
        
dataType = dataTypes[0]
interface = interfaces[0]

# Define functions for data access

In [4]:
def getNetworkData(link, interface, scopeToken):
    with urllib.request.urlopen(link + "?token=" + scopeToken + "&humanReadable=False&interface=" + interface) as response:
        links = []
        text = response.read()
        text = text.decode("utf8", 'ignore')
        textDic = json.loads(text)
        return textDic[0]

In [5]:
def resetData(button = None):
    global data
    global dataLock
    global lastData
    global startTime
    
    lastData = getNetworkData(link= baseurl + networkPath + dataType, interface = interface, scopeToken=scopeToken)
    startTime = time.time()
    with dataLock:
        data = {"time":  [0],
                "trans": [0],
                "rec":   [0]}
        with output:
            output.clear_output()
        init_plot()

In [6]:
def addData():
    global data
    global dataLock
    global lastData
    
    newData = getNetworkData(link= baseurl + networkPath + dataType, interface = interface, scopeToken=scopeToken)
    with dataLock:
        data["time"].append(time.time() - startTime)
        timeDiff = data["time"][-1] - data["time"][-2]
        data["trans"].append((newData["transmitted"][dataType] - lastData["transmitted"][dataType]) / timeDiff)
        data["rec"].append((newData["received"][dataType] - lastData["received"][dataType]) / timeDiff)
        
        if len(data["time"]) > max_members:
            del data["time"][0]
            del data["trans"][0]
            del data["rec"][0]
    lastData = newData
    

# Setup the different needed widgets

In [7]:
dropdownInterface = widgets.Dropdown(
    options=interfaces,
    description='Interface:',
    style={'description_width': '50pt'}
)

def change_interface(change):
    global interface
    interface = change['new']
    resetData()
        
dropdownInterface.observe(change_interface, 'value')

In [8]:
dropdownDataType = widgets.Dropdown(
    options=dataTypes,
    description='DataType:',
    style={'description_width': '50pt'}
)

def change_dataType(change):
    global dataType
    dataType = change['new']
    resetData()
        
dropdownDataType.observe(change_dataType, 'value')

In [9]:
reset_button = widgets.Button(
    description='Reset',
    button_style='warning',
    tooltip='Reset data',
    icon='refresh',
    disabled=False
)

pause_button = widgets.Button(
    description='Pause',
    button_style='warning',
    tooltip='Pause',
    icon='pause',
    disabled=False
)

unpause_button = widgets.Button(
    description='Unpause',
    button_style='success', 
    tooltip='Unpause',
    icon='play',
    disabled=True
)

def pause(button):
    global paused
    pause_button.disabled = True
    unpause_button.disabled = False
    paused = True

def unpause(button):
    global paused
    pause_button.disabled = False
    unpause_button.disabled = True
    paused = False

line_button = widgets.Button(
    description='Line plot',
    button_style='warning',
    tooltip='plot using a line',
    disabled=True
)

scatter_button = widgets.Button(
    description='Scatter plot',
    button_style='success', 
    tooltip='plot using a scatter plot',
    disabled=False
)

def line(button):
    global linePlot
    line_button.disabled = True
    scatter_button.disabled = False
    linePlot = True
    update_scatter_chart()

def scatter(button):
    global linePlot
    line_button.disabled = False
    scatter_button.disabled = True
    linePlot = False
    update_scatter_chart()

reset_button.on_click(resetData)
pause_button.on_click(pause)
unpause_button.on_click(unpause)
line_button.on_click(line)
scatter_button.on_click(scatter)



buttons = widgets.HBox([reset_button, widgets.VBox([unpause_button, pause_button]), widgets.VBox([line_button, scatter_button])]);

In [10]:
widget_controls = widgets.HBox(children=[buttons, widgets.VBox([dropdownInterface, dropdownDataType])],
                               layout={'margin': '4px', 'justify_content': 'space-between'});

# Setup data plot with matplotlib

In [11]:
def init_plot():
    global fig
    
    with output:
        with plt.style.context("ggplot"):
            fig = plt.figure(figsize=(11, 6), dpi=80)
            fig.canvas.toolbar_visible = False
            fig.canvas.header_visible = False
            fig.canvas.footer_visible = False

            plt.xlabel("Time [s]")
            plt.ylabel(dataType + " [1/s]")
            plt.title(interface)
            plt.xlim([-1.0 * (max_members-1) * timeStep, 0])

resetData()

In [12]:
def update_scatter_chart():
    global dataLock
    
    with dataLock:
        with output:
            x_axis = []
            lastTime = data["time"][-1]
            for time in data["time"]:
                x_axis.append(time-lastTime)
            with plt.style.context("ggplot"):
                fig.clear()
                if linePlot:
                    plt.plot(x_axis,
                        data["trans"],
                        label = "transmitted"
                       )

                    plt.plot(x_axis,
                        data["rec"],
                        label = "recived"
                       )
                else:
                    plt.scatter(x = x_axis,
                        y = data["trans"],
                        s=20,
                        label = "transmitted"
                       )

                    plt.scatter(x = x_axis,
                        y = data["rec"],
                        s=20,
                        label = "recived"
                       )

                plt.xlabel("Time [s]")
                plt.ylabel(dataType + " [1/s]")
                plt.title(interface)
                plt.legend(loc="upper left")
                plt.xlim([-1.0 * (max_members-1) * timeStep, 0])
                fig.canvas.draw()

# Add everything to final output widget

In [13]:
widgets.VBox(children=[widget_controls, output], layout={'height': '600px', 'max_width': '900px'})

# Start background thread to retrieve data

In [14]:
if "dataThreadRunning" not in vars():
    dataThreadRunning = False

def updateData():
    global startTime
    global dataThreadRunning
    
    dataThreadRunning = True
    
    while True:
        while paused:
            time.sleep(0.25)
        runningSince = time.time() - startTime
        if runningSince < timeStep:
            time.sleep(timeStep - runningSince)
        addData()
        time.sleep(timeStep)
        update_scatter_chart()
    
thread = threading.Thread(target=updateData, args=())
thread.start()